In [ ]:
using Pkg
using LightGraphs
using IterTools
using DataStructures
using Base

#use Pkg.add("<<name>>") if you don't have a particular package

In [ ]:

g = Grid([2,2])
add_edge!(g,2,3)
rem_vertex!(g,1)
rem_vertex!(g,1)
for o in edges(g)
    print(o)
end

In [ ]:
function valid_omino(omino)
    if 2 in omino
        return false
    end
    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = zeros(Int8, n1,n2)
    _f = false
    g = Grid([n1,n2])
    for i=n1:-1:1
        for j=n2:-1:1
           if omino[i,j] ==0
                rem_vertex!(g,(i-1)*n1+j)
            end
            
            
            
            
            
        end   
    end
    
    return length(connected_components(g)) == 1
            
    
end



function make_omino(inds,dim)
    
    p = zeros(Int8,dim[1],dim[2])
    for loc in inds
        p[loc[1],loc[2]]=1
    end
    if valid_omino(p)
        return p
    else
        return zeros(Int8,dim[1],dim[2])
    end

end


function make_omino_set(cells,grid)
    
   pair_idx = []
    for i=1:grid[1]
        for j=1:grid[2]
            push!(pair_idx, (i,j))
        end
    end
    ominos = []
    for t in subsets(pair_idx, Val{cells}())
        p = make_omino(t,grid)
        
        if sum(p)!=0
            push!(ominos,p)
        end
    end
    
    
    return ominos
        
    
    
    
    
end


In [394]:
function biggen(omlist,grid,bad)
    print("CALLED BIGGEN\n")
    new_oms = Set()
    for o in omlist
        
        for i = 1:grid[1]
            for j = 1:grid[2]
                z = zeros(Int8,grid[1],grid[2])
                z[i,j]+=1
                if o[i,j]==0 && valid_omino(z+o) && check_holes(z+o,bad)
                    push!(new_oms,z+o)
 
                    
                end
            end
        end
    
        
        
        
        
        
    end
       
    return new_oms
end

biggen (generic function with 1 method)

In [ ]:
###
if length(new_oms)==0
    push!(new_oms,z+o)
else
    seen = false
    for x in new_oms
        if !seen && !(1 in x-z+o)
            seen = true
        end
    end
    if !seen
        push!(new_oms,z+o)
    end
end
  ###

In [396]:
function check_holes(omino, bad_holes)
                    print(omino,' ',typeof(omino),"type\n")

    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = ones(Int8, n1,n2)-omino
    _f = false
    g = Grid([n1,n2])
    for i=n1:-1:1
        for j=n2:-1:1
           if check[i,j] ==0
                rem_vertex!(g,(i-1)*n1+j)
            end
            
            
            
            
            
        end   
    end
    
    for c in connected_components(g)
        if length(c) in bad_holes
            return false
        end
    end
    return true
            
    
    
end

check_holes (generic function with 1 method)

In [ ]:
function first_zero(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==0
                return (i,j)
            end
        end
    end
    return nothing
    
end
function first_one(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==1
                return (i,j)
            end
        end
    end
    return nothing
    
end

In [ ]:
print(first_one([1 1 1 0; 1 1 0 0; 0 0 0 0; 0 0 0 0]))
s= [1 1 1 0; 1 1 0 0; 0 0 0 0; 0 0 0 0]

In [ ]:
function build_conflicts(omlist, bad_holes,num_parts)
    
    
    firstdict = DefaultDict(Set)
    
    for o in 1:length(omlist)
        k = first_one(omlist[o])
        push!(firstdict[k],o)
    end
    
    
    
    
    
    conflict = DefaultDict(Set)
    for i=1:length(omlist)
        for j=i+1:length(omlist)
            if 2 in omlist[i]+omlist[j]
                push!(conflict[i],j)
                push!(conflict[j],i)
            elseif !check_holes(omlist[i]+omlist[j],bad_holes)
                push!(conflict[i],j)
                push!(conflict[j],i)
            end
        end
    end
    return firstdict,conflict
end

In [ ]:
f,c= build_conflicts(make_omino_set(3,[3,3]),[],2)

In [ ]:
function get_next(fd, curr_inds, omlist)
    
   return [ d for d in fd[first_zero( sum([omlist[j] for j in curr_inds    ])     )] if verify(curr_inds + d)   ] 
    
end
            

In [389]:
function enumerator(grid, om_sizes,num_parts)
    count = 0
    oms = []
    bad_hole_sizes = [ [i for i = 1:minimum(om_sizes)-1]      ;[i for i = maximum(om_sizes)+1:2*minimum(om_sizes)-1]        ]
    
    tmp_oms = make_omino_set(1,grid)
    
    if 1 in om_sizes
        oms = [oms;tmp_oms]
    end
    
    for sz = 2:maximum(om_sizes)
        tmp_oms = biggen(tmp_oms,grid,bad_hole_sizes)
        if sz in om_sizes
            oms = [oms;tmp_oms]
        end
    end
    
    
    oms = [ o for o in oms if check_holes(o,bad_hole_sizes)]
    print("MADE OMINOS\n")
    fd,cf = build_conflicts(oms,bad_hole_sizes,num_parts)
    print("MADE DICTIONARIES\n")           
    
    function verify(plan)
        #print(plan,'\n')
        for p in subsets(plan,Val{2}())
            if p[2] in cf[p[1]]
                return false
            end
        end
        return true
    end
                
                
    function recurs_part(plan,steps)
        #print("plan:",plan,"  sum: ",sum([oms[j] for j in plan]),'\n')
        if steps == 0
            count += sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2]
            return
        
        elseif sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2]
        return
        end
        for q in fd[first_zero( sum([oms[j] for j in plan     ]))]
            if verify([plan;[q]])
                recurs_part([plan;[q]],steps-1)
            end
        end
    end
    
    #print(fd,'\n')
    for p in fd[(1,1)]
        print(" ",p," ")
        recurs_part([p],num_parts-1)
    end
    
    
   print('\n') 
   return count 
    
end

enumerator (generic function with 1 method)

In [397]:
@time enumerator([4,4],[4],4)




CALLED BIGGEN
Int8[1 1 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[1 0 0 0; 1 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[1 1 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 1 0; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 0 0; 0 1 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 1 0; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 1; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 0; 0 0 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 1; 0 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 1; 0 0 0 1; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[1 0 0 0; 1 0 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 0 0 0; 1 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 0 0; 0 1 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 0 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 0 0; 0 1 0 0; 0 0 0 0] A

Int8[0 0 0 0; 0 0 1 0; 0 1 1 0; 0 1 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 1 1 1 0; 0 1 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 1 1 1; 0 1 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 1 1 0; 1 1 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 1 1 0; 0 1 1 0] Array{Int8,2}type
Int8[0 1 0 0; 0 1 1 0; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 0; 0 1 1 0; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 1 0; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 1 1; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 1 0; 1 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 1 0; 0 1 1 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 1 1 0; 0 1 0 0; 0 1 0 0] Array{Int8,2}type
Int8[0 0 0 0; 0 0 1 0; 0 0 1 0; 0 0 1 1] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 1 1 0; 0 0 1 1] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 0 1 1; 0 0 1 1] Array{Int8,2}type
Int8[0 0 0 0; 0 0 0 0; 0 0 1 0; 0 1 1 1] Array{Int8,2}type
Int8[0 1 0 0; 0 1 0 0; 0 1 0 0; 0 1 0 0] Array{Int8,2}ty

Int8[1 1 0 0; 0 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[1 1 0 0; 0 1 0 0; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[1 0 0 0; 1 0 0 0; 1 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 0 0; 1 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 0 0 0; 1 1 1 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 0 0 0; 1 1 0 0; 1 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 0 0 0; 1 1 0 0; 0 1 0 0] Array{Int8,2}type
Int8[1 0 0 0; 1 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 0 0; 1 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 0; 1 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 1 1; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 1 0; 1 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 1 0; 0 1 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 0 0; 1 1 1 0; 0 0 1 0; 0 0 0 0] Array{Int8,2}type
Int8[0 1 1 0; 0 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 1; 0 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}type
Int8[0 0 1 0; 1 1 1 0; 0 0 0 0; 0 0 0 0] Array{Int8,2}ty

MethodError: MethodError: no method matching size(::Set{Any})
Closest candidates are:
  size(!Matched::BitArray{1}) at bitarray.jl:70
  size(!Matched::BitArray{1}, !Matched::Any) at bitarray.jl:74
  size(!Matched::Core.Compiler.StmtRange) at show.jl:1561
  ...